## The gioi di dong



In [9]:
import bs4
import pandas as pd
import requests


def getSoup(url):
  # dùng header của postman để không bị từ chối 
  headers={'User-Agent':'PostmanRuntime/7.29.0'}
  page = requests.get(url,headers=headers)
  return bs4.BeautifulSoup(page.text,"html.parser")


In [ ]:
mainURL="https://www.thegioididong.com"
url="https://www.thegioididong.com/Category/FilterProductBox?c=42&o=9&pi="

def get_link_from_json(url):
  headers={'User-Agent':'PostmanRuntime/7.29.0'}
  page = requests.get(url,headers=headers)
  listproducts=page.json()['listproducts']
  soup=bs4.BeautifulSoup(listproducts,"html.parser")
  links = soup.findAll('a', class_='main-contain')
  return [link.get('href') for link in links]


allLinks=[]
for i in range(5):
  allLinks+=get_link_from_json(url+str(i))

In [ ]:
len(allLinks)

98

In [ ]:
tgdd=[]
for (i,link) in enumerate(allLinks):
  try:
    print(i,mainURL+link)
    soup=getSoup(mainURL+link)
    row=[]
    row.append(soup.h1.text)
    specs=soup.find('ul', class_='parameter__list')
    lis=specs.findAll('li')
    for li in lis[:9]:
      content=""
      for span in li.span:
        content+=span.string
      row.append(content)
    row.append(soup.find('p',class_='box-price-present').string)
    tgdd.append(row)
  except Exception as e:
    print('error: ',i, e)
    continue
tgdd


0 https://www.thegioididong.com/dtdd/samsung-galaxy-s22-ultra?src=osp
1 https://www.thegioididong.com/dtdd/iphone-11?src=osp
2 https://www.thegioididong.com/dtdd/oppo-reno7-z?src=osp
3 https://www.thegioididong.com/dtdd/realme-c35?src=osp
4 https://www.thegioididong.com/dtdd/xiaomi-11t?src=osp
5 https://www.thegioididong.com/dtdd/samsung-galaxy-a03?src=osp
6 https://www.thegioididong.com/dtdd/xiaomi-redmi-note-11-pro-4g?src=osp
7 https://www.thegioididong.com/dtdd/xiaomi-redmi-note-11s-5g?src=osp
8 https://www.thegioididong.com/dtdd/iphone-13-pro-max?src=osp
9 https://www.thegioididong.com/dtdd/samsung-galaxy-a52s-5g?src=osp
10 https://www.thegioididong.com/dtdd/vivo-y15s-2021?src=osp
11 https://www.thegioididong.com/dtdd/oppo-a55-4g?src=osp
12 https://www.thegioididong.com/dtdd/samsung-galaxy-z-fold-3?src=osp
13 https://www.thegioididong.com/dtdd/oppo-find-x5-pro?src=osp
14 https://www.thegioididong.com/dtdd/iphone-13-pro?src=osp
15 https://www.thegioididong.com/dtdd/iphone-12-pro-256

[['Điện thoại Samsung Galaxy S22 Ultra 5G 128GB',
  'Dynamic AMOLED 2X',
  'Android 12',
  'Chính 108 MP & Phụ 12 MP, 10 MP, 10 MP',
  '40 MP',
  'Snapdragon 8 Gen 1 8 nhân',
  '8 GB',
  '128 GB',
  '2 Nano SIM hoặc 1 Nano SIM + 1 eSIM',
  '5000 mAh',
  '28.990.000₫ '],
 ['Điện thoại iPhone 11 64GB',
  'IPS LCD',
  'iOS 15',
  '2 camera 12 MP',
  '12 MP',
  'Apple A13 Bionic',
  '4 GB',
  '64 GB',
  '1 Nano SIM & 1 eSIM',
  '3110 mAh',
  '13.790.000₫ *'],
 ['Điện thoại OPPO Reno7 Z 5G',
  'AMOLED',
  'Android 11',
  'Chính 64 MP & Phụ 2 MP, 2 MP',
  '16 MP',
  'Snapdragon 695 5G 8 nhân',
  '8 GB',
  '128 GB',
  '2 Nano SIM (SIM 2 chung khe thẻ nhớ)',
  '4500 mAh',
  '10.490.000₫ '],
 ['Điện thoại Realme C35 64GB',
  'IPS LCD',
  'Android 11',
  'Chính 50 MP & Phụ 2 MP, 0.3 MP',
  '8 MP',
  'Unisoc T616 8 nhân',
  '4 GB',
  '64 GB',
  '2 Nano SIM',
  '5000 mAh',
  '4.290.000₫ '],
 ['Điện thoại Xiaomi 11T 5G 128GB',
  'AMOLED',
  'Android 11',
  'Chính 108 MP & Phụ 8 MP, 5 MP',
  '16 MP'

In [ ]:
len(tgdd)

96

In [ ]:
df = pd.DataFrame(tgdd, columns =["Name","Screen","OS","Rear camera","Front camera","Chip","RAM","ROM","SIM","Batery","Price"]) 
df.to_csv("thegioididong.csv")

## FPT shop
Chưa crawl được

In [ ]:
import lxml
mainURL="https://fptshop.com.vn/"
soup = bs4.BeautifulSoup(open("fptshop.html", encoding="utf8"), features="lxml")

links = soup.findAll('a',class_="cdt-product__name")
links=[link.get('href') for link in links]


FileNotFoundError: ignored

In [ ]:
url="https://fptshop.com.vn/dien-thoai/tecno-spark-7t"
headers={'User-Agent':'PostmanRuntime/7.29.0'}
page = requests.get(url,headers=headers)
soup= bs4.BeautifulSoup(page.text,"html.parser")

table=soup.find('table',class_="st-pd-table")
print(table)

## CellphoneS 

In [ ]:


def get_link_from_json_CPS(url):
  headers={'User-Agent':'PostmanRuntime/7.29.0'}
  page = requests.get(url,headers=headers)
  listproducts=page.json()
  links=[product['url'] for product in listproducts if float(product['final_price']) >0]
  return links

links=[]
for i in range(25):
  print(i)
  url="https://cellphones.com.vn/lapi/LoadMoreProductCate/index/?page={}&id=3&order=view_count2&dir=desc&fearture=flashsale_samsung".format(i)
  links+=get_link_from_json_CPS(url)
links


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


['https://cellphones.com.vn/samsung-galaxy-a73.html',
 'https://cellphones.com.vn/iphone-11.html',
 'https://cellphones.com.vn/iphone-13-pro-max.html',
 'https://cellphones.com.vn/xiaomi-redmi-note-11.html',
 'https://cellphones.com.vn/iphone-12-pro-max.html',
 'https://cellphones.com.vn/samsung-galaxy-s22-ultra.html',
 'https://cellphones.com.vn/xiaomi-redmi-note-11-pro-plus.html',
 'https://cellphones.com.vn/samsung-galaxy-s22-plus.html',
 'https://cellphones.com.vn/xiaomi-mi-11t.html',
 'https://cellphones.com.vn/samsung-galaxy-note-20-ultra-5g.html',
 'https://cellphones.com.vn/iphone-13.html',
 'https://cellphones.com.vn/iphone-12.html',
 'https://cellphones.com.vn/oppo-reno7-z.html',
 'https://cellphones.com.vn/tecno-pova-neo.html',
 'https://cellphones.com.vn/iphone-11-128gb.html',
 'https://cellphones.com.vn/xiaomi-mi-11-lite.html',
 'https://cellphones.com.vn/vivo-t1x.html',
 'https://cellphones.com.vn/xiaomi-redmi-note-11-pro.html',
 'https://cellphones.com.vn/samsung-galaxy-

In [ ]:
len(links)

274

In [ ]:
mappingCS={
    'Kích thước màn hình': 0,
    'Công nghệ màn hình': 1,
    'Camera sau': 2,
    'Camera trước': 3,
    'Chipset': 4,
    'Dung lượng RAM': 5,
    'Bộ nhớ trong': 6,
    'Pin': 7,
    'Thẻ SIM': 8,
    'Hệ điều hành': 9,
    'Độ phân giải màn hình': 10,
    'Tính năng màn hình': 11,
    'Loại CPU': 12,
    'Trọng lượng': 13,
    'Bluetooth': 14,
    'Tần số quét': 15,
    }

In [ ]:
cellphoneS=[]
for (i,link) in enumerate(links):
  try:
    print(i,link)
    row=[None for i in range(16)]
    soup=getSoup(link)
    
    table=soup.find('table',id="tskt")
    trs=table.findAll('tr')
    for tr in trs:
      ths=tr.findAll('th')
      row[mappingCS[ths[0].text.strip()]]=ths[1].text.strip()
    row.insert(0,soup.find('h1').text.strip())
    row.append(soup.find('p',class_='special-price').text)
    cellphoneS.append(row)
  except Exception as e:
    print('error: ',i, e)
    continue



0 https://cellphones.com.vn/samsung-galaxy-a73.html
1 https://cellphones.com.vn/iphone-11.html
2 https://cellphones.com.vn/iphone-13-pro-max.html
3 https://cellphones.com.vn/xiaomi-redmi-note-11.html
4 https://cellphones.com.vn/iphone-12-pro-max.html
5 https://cellphones.com.vn/samsung-galaxy-s22-ultra.html
6 https://cellphones.com.vn/xiaomi-redmi-note-11-pro-plus.html
7 https://cellphones.com.vn/samsung-galaxy-s22-plus.html
8 https://cellphones.com.vn/xiaomi-mi-11t.html
9 https://cellphones.com.vn/samsung-galaxy-note-20-ultra-5g.html
10 https://cellphones.com.vn/iphone-13.html
11 https://cellphones.com.vn/iphone-12.html
12 https://cellphones.com.vn/oppo-reno7-z.html
13 https://cellphones.com.vn/tecno-pova-neo.html
14 https://cellphones.com.vn/iphone-11-128gb.html
15 https://cellphones.com.vn/xiaomi-mi-11-lite.html
16 https://cellphones.com.vn/vivo-t1x.html
17 https://cellphones.com.vn/xiaomi-redmi-note-11-pro.html
18 https://cellphones.com.vn/samsung-galaxy-a73.html
19 https://cellpho

In [ ]:
df = pd.DataFrame(cellphoneS, columns =["Name","Screen size","Screen type","Rear camera","Front camera","Chip","RAM","ROM","Batery","SIM","OS","Screen resolution","Screen feature","CPU type","Weight","Bluetooth","Refesh rate","Price"]) 
df


,Name,Screen size,Screen type,Rear camera,Front camera,Chip,RAM,ROM,Batery,SIM,OS,Screen resolution,Screen feature,CPU type,Weight,Bluetooth,Refesh rate,Price
0,Samsung Galaxy A73 (5G),6.7 inches,Super AMOLED,"Camera chính: 108 MP, f/1.8, PDAF, OIS Camera...","32 MP, f/2.2",Snapdragon 778G 5G 8 nhân,8 GB,256 GB,5000 mAh,2 SIM (Nano-SIM),"Android 12, One UI 4.1",1080 x 2400 pixels (FullHD+),"Tần số quét 120 Hz, Kính cường lực Corning Gor...",Octa-core (2.4 GHz & 1.8 GHz),181 g,v5.0,None,10.190.000 ₫
1,iPhone 11 64GB I Chính hãng VN/A,6.1 inches,IPS LCD,Camera kép 12MP:- Camera góc rộng: ƒ/1.8 apert...,"12 MP, ƒ/2.2 aperture",A13 Bionic,4 GB,64 GB,3110 mAh,Nano-SIM + eSIM,iOS 13 hoặc cao hơn (Tùy vào phiên bản phát hành),1792 x 828 pixel,True-tone,Hexa-core,194 g,5.0,60Hz,11.490.000 ₫
2,iPhone 13 Pro Max 128GB | Chính hãng VN/A,6.7 inches,OLED,"Camera góc rộng: 12MP, ƒ/1.5 Camera góc siêu ...","12MP, ƒ/2.2",Apple A15,6 GB,128 GB,"4,325mAh",2 SIM (nano‑SIM và eSIM),iOS15,2778 x 1284 pixel,"120Hz, Super Retina XDR với ProMotion 6.1‑inch...",None,240g,v5.0,120Hz,29.790.000 ₫
3,Xiaomi Redmi Note 11,6.43 inches,AMOLED,"Camera góc rộng: 50 MP, f/1.8, PDAF Camera gó...","13 MP, f/2.5",Snapdragon 680 8 nhân,4 GB,64 GB,Li-Po 5000 mAh,2 SIM (Nano-SIM),"Android 11, MIUI 12.5",None,None,None,None,None,None,4.490.000 ₫
4,iPhone 12 Pro Max 128GB I Chính hãng VN/A,6.7 inches,OLED,"Camera chính: 12 MP, f/1.6, 26mm, 1.4µm, dual ...","12 MP, f/2.2, 23mm (wide), 1/3.6"" SL 3D, (dept...",Apple A14 Bionic (5 nm),6 GB,128 GB,"Li-Ion, sạc nhanh 20W, sạc không dây 15W, USB ...",2 SIM (nano‑SIM và eSIM),iOS 14.1 hoặc cao hơn (Tùy vào phiên bản phát ...,1284 x 2778 pixels,HDR10Dolby VisionTrue-toneĐộ sáng 800 nits,Hexa-core,228,"5.0, A2DP, LE",60Hz,27.000.000 ₫
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,Samsung Galaxy A20s,6.5 inches,None,- 13 MP (F1.8) + 8 MP (F2.2) + 5 MP (F2.2) - Đ...,"- 8 MP (F2.0) - Tự động lấy nét, Quay video H...",Qualcomm Snapdragon 450,3 GB,32 GB,"4000 mAh, công nghệ sạc nhanh siêu tốc 15W",2 SIM (Nano-SIM),Android 9.0 (Pie),None,None,"8 nhân 64-bit, 1.8 GHz",169 g,"v5.0, A2DP, LE",None,3.550.000 ₫
263,Vivo Y20 (2021),6.5 inches,IPS LCD,"13 MP, f/1.8, (wide), PDAF 2 MP, f/2.4, (macro...","8 MP, (wide)",MediaTek Helio P35,4 GB,64 GB,Li-Po 5000 mAh,2 SIM (Nano-SIM),"Android 10, Funtouch 10.5",720 x 1600 pixel,None,None,192.3 g,None,None,3.350.000 ₫
264,Samsung Galaxy A8 (2018) Chính hãng,5.6 inches,None,"16 MP, f/1.7, tự động lấy nét nhận diện theo ...","16 MP + 8 MP, f/1.9, 1080p",Samsung Exynos 7 Octa 7885,None,32 GB,Li-ion 3000 mAh,2 SIM (Nano-SIM),7.1.1 (Nougat),1080 x 2220 pixels (FullHD+),None,2x 2.2 GHz ARM Cortex-A73 & 6x 1.6 GHz ARM Cor...,172 g (6.07 oz),"5.0, A2DP, EDR, LE",None,8.090.000 ₫
265,Tecno Spark 8C 4GB 128GB,6.6 inches,IPS LCD,13 MP + 8MP,8 MP,UNISOC T606,4GB + Mở rộng 3GB,64 GB,Li-Po 5000 mAh,2 SIM (Nano-SIM),"Android 11 (Go edition), HiOS 7.6",720 x 1612 (HD+),Tần số quét 90Hz Tỷ lệ 20:9 (mật độ ~ 267 ppi),None,None,Có,90Hz,2.890.000 ₫


In [ ]:
df.to_csv("cellphoneS.csv")

## hoanghamobile

In [ ]:
mainURL="https://hoanghamobile.com"
url="https://hoanghamobile.com/dien-thoai-di-dong?p=8"
soup=getSoup(url)
links=soup.findAll('div',class_="promote")
links=[mainURL+link.a.get('href') for link in links]




In [ ]:
mapping={
      'Công nghệ màn hình::':0,
      'Độ phân giải::':1,
      'Màn hình rộng::':2,
      'Độ phân giải:':3,
      'Hệ điều hành:':4,
      'Chip xử lý (CPU):':5,
      'Bộ nhớ trong (ROM):':6,
      'RAM:':7,
      'Mạng di động:':8,
      'Số khe sim:':9,
      'Dung lượng pin:':10,
}

In [ ]:
# file 1 từ 0->346 thì server bị lỗi , chạy lần 2 từ 347 đến hết
hoangHa=[]
for (i,link) in enumerate(links[347:]):
  try:
    print(i+347,link)
    soup=getSoup(link)
    specs={s.strong.text.strip():s.span.text.strip() for s in soup.find('div','specs-special').findAll('li') }
    row=[None for i in range(11)]
    for (key,val) in specs.items():
      row[mapping[key]]=val
    name=soup.find('div','top-product').find('h1').text.strip()
    row.insert(0,name)
    price=soup.find('p',class_='current-product-price').strong.text.strip()
    row.append(price) 
    hoangHa.append(row)
  except Exception as e:
    print('error: ',i, e)
    continue


347 https://hoanghamobile.com/dien-thoai-di-dong/apple-iphone-13-pro-max-512gb-chinh-hang-vn-a
348 https://hoanghamobile.com/dien-thoai-di-dong/apple-iphone-13-pro-max-1TB-chinh-hang-vn
349 https://hoanghamobile.com/dien-thoai-di-dong/iphone-11-64gb-chinh-hang-vn-a
350 https://hoanghamobile.com/dien-thoai-di-dong/iphone-11-128gb-chinh-hang-vn-a
351 https://hoanghamobile.com/dien-thoai-di-dong/apple-iphone-12-128gb-chinh-hang-vn-a
352 https://hoanghamobile.com/dien-thoai-di-dong/apple-iphone-12-256gb-chinh-hang-vn-a
353 https://hoanghamobile.com/dien-thoai-di-dong/apple-iphone-12-64gb-chinh-hang-vn-a
354 https://hoanghamobile.com/dien-thoai-di-dong/oppo-a55-chinh-hang
355 https://hoanghamobile.com/dien-thoai-di-dong/oppo-a95-chinh-hang
356 https://hoanghamobile.com/dien-thoai-di-dong/oppo-a16-3g-32g-chinh-hang
357 https://hoanghamobile.com/dien-thoai-di-dong/apple-iphone-12-mini-64gb-chinh-hang-vn-a
358 https://hoanghamobile.com/dien-thoai-di-dong/apple-iphone-12-mini-128gb-chinh-hang-v

File hoangHa1.csv từ dòng 1->346

File hoangHa2.csv từ dòng 347-> 1263

In [ ]:
df = pd.DataFrame(hoangHa, columns =["Name","Screen type","Screen resolution","Screen size","Camera","OS","Chip","ROM","RAM","Mobile network","SIM","Batery","Price"]) 
df.to_csv("hoangHa2.csv")


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 911 entries, 0 to 910
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name               911 non-null    object
 1   Screen type        875 non-null    object
 2   Screen resolution  785 non-null    object
 3   Screen size        442 non-null    object
 4   Camera             779 non-null    object
 5   OS                 732 non-null    object
 6   Chip               719 non-null    object
 7   ROM                785 non-null    object
 8   RAM                803 non-null    object
 9   Mobile network     680 non-null    object
 10  SIM                719 non-null    object
 11  Batery             710 non-null    object
 12  Price              911 non-null    object
dtypes: object(13)
memory usage: 92.6+ KB


## nguyenkim.com

In [10]:
linksNK=[]
for i in range(1,8):
  soup=getSoup("https://www.nguyenkim.com/dien-thoai-di-dong/page-{}/".format(i))
  links=soup.findAll('div',class_="product-title")
  links=[link.a.get('href') for link in links]
  linksNK=linksNK+links

linksNK

['https://www.nguyenkim.com/dien-thoai-iphone-12-64gb-xanh-la.html',
 'https://www.nguyenkim.com/dien-thoai-iphone-12-128gb-do.html',
 'https://www.nguyenkim.com/dien-thoai-iphone-12-pro-256gb-xam.html',
 'https://www.nguyenkim.com/dien-thoai-iphone-12-pro-256gb-xanh.html',
 'https://www.nguyenkim.com/iphone-13-pro-128gb-bac.html',
 'https://www.nguyenkim.com/iphone-13-pro-256gb-xam.html',
 'https://www.nguyenkim.com/iphone-13-pro-256gb-xanh-duong.html',
 'https://www.nguyenkim.com/dien-thoai-iphone-13-pro-max-128gb-vang-dong-vi.html',
 'https://www.nguyenkim.com/dien-thoai-iphone-11-64gb-xanh-la-vi-3.html',
 'https://www.nguyenkim.com/dien-thoai-iphone-11-64gb-trang-vi-2.html',
 'https://www.nguyenkim.com/dien-thoai-xiaomi-redmi-9a-2gb-32gb-xanh-duong.html',
 'https://www.nguyenkim.com/dien-thoai-samsung-galaxy-a52-4g-8gb-128gb-xanh.html',
 'https://www.nguyenkim.com/dien-thoai-samsung-galaxy-a32-4g-6gb-128gb-xanh.html',
 'https://www.nguyenkim.com/dien-thoai-samsung-galaxy-a52-4g-8gb

In [ ]:
len(linksNK)

209

In [ ]:
soup=getSoup(linksNK[0])
tb=soup.find("tbody",class_="popup")

price=soup.find('span',class_="nk-price-final").text



'16.990.000đ'

In [21]:
mapping={
      'Loại màn hình:':0,
      'Kích thước màn hình:':1,
      "Độ phân giải màn hình:": 2,
      'Camera trước:':3,
      'Camera sau:':4,
      'Hệ điều hành:':5,
      'Chipset:':6,
      'Bộ nhớ trong:':7,
      'RAM:':8,
      'Dung lượng Pin:':9,
      
}

In [22]:
nguyenKim=[]
for (i,link) in enumerate(linksNK):
  try:
    print(i,link)
    row=[None for i in range(10)]
    soup=getSoup(link)
    table=soup.find('tbody',class_="popup")
    trs=table.findAll('tr')
    for tr in trs:
      tds=tr.findAll('td')
      if tds[0].text.strip() in mapping:
        row[mapping[tds[0].text.strip()]]=tds[1].text.strip()
    row.insert(0,soup.find('h1',class_="product_info_name").text.strip())
    row.append(soup.find('span',class_="nk-price-final").text.strip())
    print(row)
    nguyenKim.append(row)
  except Exception as e:
    print('error: ',i, e)
    continue

0 https://www.nguyenkim.com/dien-thoai-iphone-12-64gb-xanh-la.html
['Điện thoại iPhone 12 64GB Xanh lá', 'Super Retina XDR', '6.1 inch "', '2532 x 1170 pixels', '12MP', '12 MP (kép)', 'iOS 14.1', 'Apple A14 Bionic', '64 GB', '4 GB', None, '16.990.000đ']
1 https://www.nguyenkim.com/dien-thoai-iphone-12-128gb-do.html
['Điện thoại iPhone 12 128GB Đỏ', 'Super Retina XDR', '6.1 inch "', '2532 x 1170 pixels', '12 MP', '12 MP (kép)', 'iOS 14.1', 'Apple A14 Bionic', '128 GB', '4 GB', None, '18.990.000đ']
2 https://www.nguyenkim.com/dien-thoai-iphone-12-pro-256gb-xam.html
['Điện thoại iPhone 12 Pro 256GB Xám', None, '6.1 "', None, '12 MP', '12 MP, 12 MP, 12 MP', 'iOS 14.1', 'Apple A14 Bionic', '256 GB', '6 GB', None, '24.990.000đ']
3 https://www.nguyenkim.com/dien-thoai-iphone-12-pro-256gb-xanh.html
['Điện thoại iPhone 12 Pro 256GB Xanh', None, '6.1 "', None, '12 MP', '12 MP, 12 MP, 12 MP', 'iOS 14.1', 'Apple A14 Bionic', '256 GB', '6 GB', None, '28.990.000đ']
4 https://www.nguyenkim.com/iphone

In [25]:
df = pd.DataFrame(nguyenKim, columns =["Name","Screen type","Screen size","Screen resolution","Front camera","Rear camera","OS","Chip","ROM","RAM","Batery","Price"]) 
df

,Name,Screen type,Screen size,Screen resolution,Front camera,Rear camera,OS,Chip,ROM,RAM,Batery,Price
0,Điện thoại iPhone 12 64GB Xanh lá,Super Retina XDR,"6.1 inch """,2532 x 1170 pixels,12MP,12 MP (kép),iOS 14.1,Apple A14 Bionic,64 GB,4 GB,None,16.990.000đ
1,Điện thoại iPhone 12 128GB Đỏ,Super Retina XDR,"6.1 inch """,2532 x 1170 pixels,12 MP,12 MP (kép),iOS 14.1,Apple A14 Bionic,128 GB,4 GB,None,18.990.000đ
2,Điện thoại iPhone 12 Pro 256GB Xám,None,"6.1 """,None,12 MP,"12 MP, 12 MP, 12 MP",iOS 14.1,Apple A14 Bionic,256 GB,6 GB,None,24.990.000đ
3,Điện thoại iPhone 12 Pro 256GB Xanh,None,"6.1 """,None,12 MP,"12 MP, 12 MP, 12 MP",iOS 14.1,Apple A14 Bionic,256 GB,6 GB,None,28.990.000đ
4,Điện thoại iPhone 13 Pro 128GB Bạc,OLED,"6.1 """,2532 x 1170 pixels,12MP,12MP-12MP-12MP,iOS 15,Apple A15 Bionic,128GB,6GB,None,27.490.000đ
...,...,...,...,...,...,...,...,...,...,...,...,...
204,Điện thoại OPPO A94 8GB/128GB Đen,AMOLED,"6.43 """,FHD+ (1080 x 2400) pixels,32 MP,48 MP + 8 MP + 2 MP + 2 MP,"ColorOS 11.1, nền tảng Android 11","Helio P95, tối đa 2.2GHz",128GB,8GB,"4310mAh (Typ), Sạc nhanh VOOC 4.0 - 30W",6.152.000đ
205,Điện thoại Samsung Galaxy S20 Ultra Đen,Dynamic AMOLED 2x,"6.9 """,None,40 MP,108 MP + 48 MP + 12 MP,"Android 10, One UI 2.1",Exynos 990,128 GB,12 GB,5000 mAh,23.992.000đ
206,Samsung Galaxy S20+ Đen,Dynamic AMOLED 2x,"6.7 """,None,10 MP,4 MP + 12 MP+ 12 MP,"Android 10, One UI 2.1",Exynos 990,128 GB,8 GB,4500 mAh,23.990.000đ
207,Điện thoại Samsung Galaxy A52 4G 8GB/128GB Tím,Màn hình tràn viền vô cực Super AMOLED Full HD...,"6.5 """,None,32MP,4 camera: 64MP+12MP + 5MP + 5MP,None,Vi xử lý 8 nhân Snapdragon 720G,128GB,8GB,"4500mAh, Công nghệ Sạc nhanh siêu tốc 25W",9.290.000đ


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name               209 non-null    object
 1   Screen type        176 non-null    object
 2   Screen size        207 non-null    object
 3   Screen resolution  143 non-null    object
 4   Front camera       206 non-null    object
 5   Rear camera        202 non-null    object
 6   OS                 152 non-null    object
 7   Chip               193 non-null    object
 8   ROM                209 non-null    object
 9   RAM                206 non-null    object
 10  Batery             120 non-null    object
 11  Price              209 non-null    object
dtypes: object(12)
memory usage: 19.7+ KB


In [27]:
df.to_csv("nguyenkim.csv")